# OpenGridMap Notebook

# Step1. Preparation

## Import necessary modules

In [1]:
import psycopg2
import json
import ipyleaflet as L
import networkx as nx
from ipyleaflet import (
    Map,
    Marker,
    TileLayer, ImageOverlay,
    Polyline, Polygon, Rectangle, Circle, CircleMarker,
    GeoJSON,
    DrawControl
)

## Database connection

In [35]:
# Set connection to gis db
try:
    conn = psycopg2.connect("dbname=gis user=jennyzhou")
except:
    print 'Fail to connect to Postgres Server'

# Step2. Data selection

## Select area of Garching on map

In [36]:
# Select polygon of Garching
cur = conn.cursor() 
cur.execute("SELECT ST_AsGeoJSON(ST_FlipCoordinates(ST_Transform(ST_Centroid(way), 4326))), ST_AsGeoJSON(ST_FlipCoordinates(ST_Transform(way, 4326))) FROM planet_osm_polygon WHERE osm_id = -30971; ")
garching = cur.fetchall()
cur.close()

## Select transformer entries

In [37]:
cur = conn.cursor()
# Select all the transformers inside Garching
cur.execute("SELECT ST_AsGeoJSON(ST_FlipCoordinates(ST_Transform(way, 4326))) FROM student_transformers;")
transformers = cur.fetchall()
cur.close()

#cur.execute("CREATE TABLE transformer_geo (id serial PRIMARY KEY, osm_id bigint, geo_json text);")
#cur.execute("INSERT INTO transformer_geo (osm_id, geo_json) SELECT osm_id, ST_AsGeoJSON(ST_Transform(way, 4326)) FROM planet_osm_point WHERE power != ''ORDER BY name;")

## Select house entries

In [38]:
cur = conn.cursor()
# Too Slow
cur.execute("SELECT ST_AsGeoJson(ST_FlipCoordinates(ST_Transform(ST_Centroid(way), 4326))) FROM planet_osm_polygon WHERE ST_Within(way,(SELECT way FROM planet_osm_polygon WHERE osm_id = -30971)) AND building != '' LIMIT 200; ")
houses = cur.fetchall()
conn.close()

# Step3. Data visualisation with ipyleaflet

## Display area of Garching

In [41]:
for row in garching:
    garchingCentJson = json.loads(row[0])
    garchingPolJson = json.loads(row[1])
    #print garchingPolJson
    
# Initialise map with center of Garching 
map = Map(default_tiles=TileLayer(opacity=1.0),center= garchingCentJson['coordinates'],zoom =16)

# Show boundary of Garching
garchingOuter = Polygon(locations=garchingPolJson['coordinates'], weight=4,
            color='#003d99', opacity=0.8, fill_opacity=0.5,
            fill_color='#ccffcc')
#L.GeoJSON(data=garchingPolJson)
map += garchingOuter

# Display map
map

## Mark houses and transformers

In [42]:
# Mark transformers
tmarkers = []
for row in transformers:  
    #print " ", row[0]
    mk = Marker(location=json.loads(row[0])['coordinates'])
    tmarkers.append(mk)
    map.add_layer(mk)

# Mark houses
hmarkers = []
for row in houses:
    c = Circle(location=json.loads(row[0])['coordinates'], weight=5, opacity = 0.5, color = 'red', radius = 2) 
    hmarkers.append(c)
    map.add_layer(c) 


In [92]:
# Clean Markers
for mk in tmarkers:
    map.remove_layer(mk)

for mk in hmarkers:
    map.remove_layer(mk)

# Step4. Find best graph

## Build graph with data set

In [4]:
# To be implemented
def buildGraph(graph):
    # graph ["1 2 10.1", "2 3 5.3"]
    G=nx.parse_edgelist(graph, nodetype = int, data=(('weight',float),))
    return G


In [5]:
# Testing Code
graph = ["1 2 10.1", "2 3 5.3"]
G = buildGraph(graph)
print G.nodes()
print G.edges(data = True)

[1, 2, 3]
[(1, 2, {'weight': 10.1}), (2, 3, {'weight': 5.3})]


## Run network algorithm

In [ ]:
# To be implemented

# Step5. Display graph on map

# ================= Testing section [IGNORE]====================


## Map initialisation

In [10]:
ingolstadt = [48.7630164995764,11.4250395050931] 
munich = [48.1371078976613,11.5753821798641]
garching =  [11.6032985849595,48.2465396933038]
#ismaning = []

map = Map(default_tiles=TileLayer(opacity=1.0),center=munich,zoom =8)
map

## Play around with markers

In [11]:
# Normal marker
munMK = Marker(location=munich)
ingMK = Marker(location=ingolstadt)
map+=munMK
map+=ingMK


In [12]:
# Circle 
c = Circle(location=munich)
map.add_layer(c)
#c.interact(weight=(0,10,1), opacity=(0.0,1.0,0.01))

In [38]:
map.remove_layer(c)

In [13]:
# Circle Marker
cm = CircleMarker(location=ingolstadt, radius=30, weight=2,
                  color='#F00', opacity=1.0, fill_opacity=1.0,
                  fill_color='#0F0')
map.add_layer(cm)

In [41]:
map.remove_layer(cm)

## Using GeoJSON 

In [15]:
data = {"type":"Point","coordinates": ingolstadt}
layer = ll.GeoJSON(data=data)
map.add_layer(layer)

In [16]:
layer.data['coordinates']

[48.7630164995764, 11.4250395050931]